In [1]:
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from os import listdir, getcwd
from os.path import join
import matplotlib.pyplot as plt
import numpy as np
from lib import unet
from lib import generator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

Using TensorFlow backend.


In [2]:
DATA_DIR = join(getcwd(), 'data')
TRAIN_DIR = join(DATA_DIR, 'train')
MASK_DIR = join(DATA_DIR, 'train_masks')
TRAIN_IMAGE_LIST = sorted(listdir(TRAIN_DIR))
MASK_IMAGE_LIST = sorted(listdir(MASK_DIR))
IMG_WIDTH = 320

In [3]:
train_images, test_images, train_masks, test_masks = train_test_split(TRAIN_IMAGE_LIST, 
                                                                      MASK_IMAGE_LIST, 
                                                                      test_size=0.10000)
train_images, validation_images, train_masks, validation_masks = train_test_split(train_images,
                                                                                  train_masks,
                                                                                  test_size=0.11111)

In [4]:
train_generator = generator.image_generator(TRAIN_DIR, 
                                            MASK_DIR, 
                                            train_images, 
                                            train_masks, 
                                            batch_size = 5, 
                                            img_dim = [int((1280 / 1918) * IMG_WIDTH), IMG_WIDTH], 
                                            num_colors = 8)

validation_generator = generator.image_generator(TRAIN_DIR, 
                                                 MASK_DIR, 
                                                 validation_images, 
                                                 validation_masks, 
                                                 batch_size = 5, 
                                                 img_dim = [int((1280 / 1918) * IMG_WIDTH), IMG_WIDTH], 
                                                 num_colors = 8)

test_generator = generator.image_generator(TRAIN_DIR, 
                                           MASK_DIR, 
                                           test_images, 
                                           test_masks, 
                                           batch_size = 5, 
                                           img_dim = [int((1280 / 1918) * IMG_WIDTH), IMG_WIDTH], 
                                           num_colors = 8)

In [5]:
model = unet.get_unet_model([IMG_WIDTH, IMG_WIDTH, 3])

model.compile(optimizer=Adam(1e-4), 
              loss='binary_crossentropy',
              metrics=[unet.dice_coef])

In [6]:
checkpoint = ModelCheckpoint('best-fit-model.hdf5',
                             monitor='val_dice_coef',
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

lr_scheduler = LearningRateScheduler(lr_schedule)

NameError: name 'lr_schedule' is not defined

In [ ]:
hist = model.fit_generator(train_generator,
                           steps_per_epoch = 814,
                           epochs = 10,
                           validation_data = validation_generator,
                           validation_steps = 102,
                           callbacks=[checkpoint, lr_scheduler])

In [ ]:
img, mask = next(train_generator)
plt.figure(figsize=(20,20))
for i in range(img.shape[0]):
    plt.subplot(img.shape[0],2,2*i+1)
    plt.imshow(img[i,:,:,:])
    plt.subplot(img.shape[0],2,2*i+2)
    plt.imshow(mask[i,:,:,0])
    
plt.show()